<a href="https://colab.research.google.com/github/present42/PyTorchPractice/blob/main/Fluent_Python_ch15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 15. More About Type Hints
10:33 -
 - Overloaded function signatures
 - `typing.TypedDict` for type hinting `dicts` used as records
 - Type Casting
 - Runtime access to type hints
 - generic types
  - declaring a generic class
  - variance: invariant, convariant, contravariant
  - generic static protocols

## Overloaded Signatures

In [ ]:
help(sum)

Help on built-in function sum in module builtins:

sum(iterable, /, start=0)
    Return the sum of a 'start' value (default: 0) plus an iterable of numbers
    
    When the iterable is empty, return the start value.
    This function is intended specifically for use with numeric values and may
    reject non-numeric types.



In [ ]:
# mysum.py
import functools
import operator
from collections.abc import Iterable
from typing import overload, Union, TypeVar

T = TypeVar('T')
S = TypeVar('S')

@overload
def sum(it: Iterable[T]) -> Union[T, int]: ...

@overload
def sum(it: Iterable[T], /, start: S) -> Union[T, S]: ...
def sum(it, /, start=0):
  return functools.reduce(operator.add, it, start)

In [ ]:
sum([1, 2, 3])

6

In [ ]:
max([1, 2, 3])
max(1, 2, 3)
max([])

ValueError: max() arg is an empty sequence

In [ ]:
# mymax.py
from collections.abc import Callable, Iterable
from typing import Protocol, Any, TypeVar, overload, Union

class SupportsLessThan(Protocol):
  def __lt__(self, other: Any) -> bool: ...

T = TypeVar('T')
LT = TypeVar('LT', bound=SupportsLessThan)
DT = TypeVar('DT')

MISSING = object()  # unique object instance used as sentinel
EMPTY_MSG = 'max() arg is an empty sequence'

@overload
def max(__arg1: LT, __arg2: LT, *args: LT, key: None = ...) -> LT:
  ...

@overload
def max(__arg1: T, __arg2: T, *args: T, key: Callable[[T], LT] = ...) -> T:
  ...

@overload
def max(__iterable: Iterable[LT], *, key: None = ...) -> LT:
  ...

@overload
def max(__iterable: Iterable[T], *, key: Callable[[T], LT] = ...) -> T:
  ...

@overload
def max(__iterable: Iterable[LT], *, key: None = ...,
        default: DT) -> Union[LT, DT]:
  ...

@overload
def max(__iterable: Iterable[T], *, key: Callable[[T], LT] = ...,
        default: DT) -> Union[T, DT]:
  ...

def max(first, *args, key=None, default=MISSING):
  if args: # if there is a second positional argument
    series = args
    candidate = first
  else:
    series = iter(first)
    try:
      candidate = next(series)
    except StopIteration:
      if default is not MISSING:
        return default
      raise ValueError(EMPTY_MSG) from None
  if key is None:
    for current in series:
      if candidate < current:
        candidate = current
  else:
    candidate_key = key(candidate)
    for current in series:
      current_key = key(current)
      if candidate_key < current_key:
        candidate = current
        candidate_key = current_key
  return candidate

In [ ]:
def test(a, *, b, c=3):
  print(a, b, c)

In [ ]:
test(3, b=2)

3 2 3


In [ ]:
test(a=3, b=2)

TypeError: test() got some positional-only arguments passed as keyword arguments: 'a'

## TypedDict

 - It's tempting to use `TypedDict` to protect against errors while handling dynamic data structures like JSON API responses. But the example here make clear that correct handling of JSON must be done at runtime.

In [ ]:
from typing import TypedDict

class BookDict(TypedDict):
  isbn: str
  title: str
  authors: list[str]
  pagecount: int

At first glance, `typing.TypedDict` may seem like a data class builder, similar to `typing.NamedDict`.

But, it only exists for the benefit of type checkers! (No runtime effect)

In [ ]:
pp = BookDict(title="Programming Pearls",
              authors='Jon Bentley', # no type checking in runtime
              isbn='0201657880',
              pagecount=256)

In [ ]:
pp

{'title': 'Programming Pearls',
 'authors': 'Jon Bentley',
 'isbn': '0201657880',
 'pagecount': 256}

In [ ]:
type(pp) # result of calling BookDict is a plain dict

dict

In [ ]:
BookDict.__annotations__

{'isbn': str, 'title': str, 'authors': list[str], 'pagecount': int}

In [ ]:
# demo_books.py
from typing import TYPE_CHECKING

def demo() -> None: # Don't forget to add None so that Mypy doesn't ignore the fcn
  book = BookDict(
      isbn='0134757599',
      authors=['Martin Fowler', 'Kent Beck'],
      title='Refactoring, 2e',
      pagesount=478
  )
  authors = book['authors']
  if TYPE_CHECKING:
    reveal_type(authors)
  authors = 'Bob'
  book['weight'] = 4.2
  del book['title']

if __name__ == "__main__":
  demo()

In [ ]:
!pip install mypy

In [ ]:
!mypy demo_book.py

demo_book.py:14: note: Revealed type is "builtins.list[builtins.str]"
demo_book.py:15: error: Incompatible types in assignment (expression has type "str", variable has type "list[str]")  [assignment]
demo_book.py:16: error: TypedDict "BookDict" has no key "weight"  [typeddict-unknown-key]
demo_book.py:17: error: Key "title" of TypedDict "BookDict" cannot be deleted  [misc]
Found 3 errors in 1 file (checked 1 source file)


In [ ]:
# books.py
AUTHOR_ELEMENT = '<AUTHOR>{}</AUTHOR>'

def to_xml(book: BookDict) -> str: # whole point of example = using BookDict in the fcn signature
  elements: list[str] = []
  for key, value in book.items():
    if isinstance(value, list):
      elements.extend(
          AUTHOR_ELEMENT.format(n) for n in value
      )
    else:
      tag = key.upper()
      elements.append(f'<{tag}>{value}</{tag}>')
  xml = '\n\t'.join(elements)
  return f'<BOOK>\n\t{xml}\n</BOOK>'

In [ ]:
!mypy books.py

Success: no issues found in 1 source file


In [ ]:
!mypy books_any.py --disallow-any-expr

Success: no issues found in 1 source file


In [ ]:
# demo_not_book.py
from books import to_xml, from_json
from typing import TYPE_CHECKING

def demo() -> None:
  NOT_BOOK_JSON = """
    {"title": "Andromeda Strain",
     "flavor": "pistachio",
     "authors": true}
  """

  not_book = from_json(NOT_BOOK_JSON)
  if TYPE_CHECKING:
    reveal_type(not_book)
    reveal_type(not_book['authors'])

  print(not_book)
  print(not_book['flavor'])

  xml = to_xml(not_book)
  print(xml)

if __name__ == "__main__":
  demo()

In [ ]:
!mypy demo_not_book.py

demo_not_book.py:14: note: Revealed type is "TypedDict('books.BookDict', {'isbn': builtins.str, 'title': builtins.str, 'authors': builtins.list[builtins.str], 'pagecount': builtins.int})"
demo_not_book.py:15: note: Revealed type is "builtins.list[builtins.str]"
demo_not_book.py:18: error: TypedDict "BookDict" has no key "flavor"  [typeddict-item]
Found 1 error in 1 file (checked 1 source file)


In [ ]:
!python demo_not_book.py

{'title': 'Andromeda Strain', 'flavor': 'pistachio', 'authors': True}
pistachio
<BOOK>
	<TITLE>Andromeda Strain</TITLE>
	<FLAVOR>pistachio</FLAVOR>
	<AUTHORS>True</AUTHORS>
</BOOK>


Takeaway: when handling data with a dynamic structure, such as JSON or XML, TypedDict is absolutely not a replacement for data vaidation at runtime.

## Type Casting

`typing.cast()` special function provides one way to handle type checking malfunctions or incorrect type hints in code we can't fix.


> Casts are used to silence spurious type checker warnings and give the type checker a little help when it can't quite understand what is going on.



In [ ]:
from typing import cast

# it'll either return the index of a str item or raise StopIteration
def find_first_str(a: list[object]) -> str:
  index = next(i for i, x in enumerate(a) if isinstance(x, str))
  # if the last line were just return a[index]
  # Mypy would infer the return type as object
  return cast(str, a[index])

In [ ]:
find_first_str([1, 2, 'a'])

'a'

## Reading Type Hints at Runtime

In [ ]:
# clipannot.py
def clip(text: str, max_len: int = 80) -> str:

Note that annotations are evaluated by the interpreter at imort time, just as parameter default values are also evaluated

In [ ]:
from clipannot import clip

clip.__annotations__

{'text': str, 'max_len': int, 'return': str}

In [ ]:
# Beginning with Python 3.7, that's how annotations are handled in any module
# that starts with this import statement
from __future__ import annotations

In [ ]:
# clip_annot_post.py
from __future__ import annotations

def clip(text: str, max_len: int = 80) -> str:
  pass

In [ ]:
from clip_annot_post import clip

clip.__annotations__

{'text': 'str', 'max_len': 'int', 'return': 'str'}

In [ ]:
from typing import get_type_hints

get_type_hints(clip)

{'text': str, 'max_len': int, 'return': str}

Dealing with problem
 - Avoid reading `__annotations__` directly; instead, use `inspect.get_annotations` or `typing.get_type_hints`
 - Write a custom function of your own as a thin wrapper around `inspect.get_annotations` or `typing.get_type_hints` and have the rest of your codebase call that custom function, so that future changes are localized to a single function.

## Implementing a Generic Class

In [ ]:
# tombola.py

import abc

class Tombola(abc.ABC):

    @abc.abstractmethod
    def load(self, iterable):
        """Add items from an iterable."""

    @abc.abstractmethod
    def pick(self):
        """Remove item at random, returning it.

        This method should raise `LookupError` when the instance is empty.
        """

    def loaded(self):
        """Return `True` if there's at least 1 item, `False` otherwise."""
        return bool(self.inspect())

    def inspect(self):
        """Return a sorted tuple with the items currently inside."""
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(items)

In [ ]:
# lotto.py

import random

from tombola import Tombola


class LottoBlower(Tombola):

    def __init__(self, iterable):
        self._balls = list(iterable)

    def load(self, iterable):
        self._balls.extend(iterable)

    def pick(self):
        try:
            position = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from empty LottoBlower')
        return self._balls.pop(position)

    def loaded(self):
        return bool(self._balls)

    def inspect(self):
        return tuple(self._balls)

In [ ]:
# generic_lotto.py

import random

from collections.abc import Iterable
from typing import TypeVar, Generic

from tombola import Tombola

T = TypeVar('T')

# Generic class declaration often use multiple inheritance,
# because we need to subclass Generic to declare the formal type params
class LottoBlower(Tombola, Generic[T]):

  def __init__(self, items: Iterable[T]) -> None:
    self._balls = list[T](items)

  def load(self, items: Iterable[T]) -> None:
    self._balls.extend(items)

  def pick(self) -> T:
    try:
        position = random.randrange(len(self._balls))
    except ValueError:
        raise LookupError('pick from empty LottoBlower')
    return self._balls.pop(position)

  def loaded(self) -> bool:
    return bool(self._balls)

  def inspect(self) -> tuple[T, ...]:
    return tuple(self._balls)

In [ ]:
# generic_lotto_demo.py

from generic_lotto import LottoBlower

machine = LottoBlower[int](range(1, 11))

first = machine.pick()
remain = machine.inspect()

In [ ]:
# generic_lotto_error.py
from generic_lotto import LottoBlower

machine = LottoBlower[int]([1, .2])

machine = LottoBlower[int](range(1, 11))

machine.load('ABC')

In [ ]:
!pip install mypy

In [ ]:
!mypy generic_lotto_demo.py

Success: no issues found in 1 source file


In [ ]:
!mypy generic_lotto_error.py

generic_lotto_error.py:4: error: List item 1 has incompatible type "float"; expected "int"  [list-item]
generic_lotto_error.py:8: error: Argument 1 to "load" of "LottoBlower" has incompatible type "str"; expected "Iterable[int]"  [arg-type]
generic_lotto_error.py:8: note: Following member(s) of "str" have conflicts:
generic_lotto_error.py:8: note:     Expected:
generic_lotto_error.py:8: note:         def __iter__(self) -> Iterator[int]
generic_lotto_error.py:8: note:     Got:
generic_lotto_error.py:8: note:         def __iter__(self) -> Iterator[str]
Found 2 errors in 1 file (checked 1 source file)


Basic Jargon for Generic Types
 - Generic type
  : type declared with one or more type variables (e.g. `LottoBlower[T]`, `abc.Mapping[KT, VT]`)
 - Formal type parameter
  : type variables that appear in a generic type declaration.
 - Parametrized type
  : A type declared with actual type parameters.
 - Actual type parameter

## Variance

In [ ]:
# invariant.py

from typing import TypeVar, Generic

class Beverage:
  """Any beverage."""

class Juice(Beverage):
  """Any fruit juice."""

class OrangeJuice(Juice):
  """orange juice."""

T = TypeVar('T')

class BeverageDispenser(Generic[T]):
  def __init__(self, beverage: T) -> None:
    self.beverage = beverage

  def dispense(self) -> T:
    return self.beverage

def install(dispenser: BeverageDispenser[Juice]) -> None:
  """ Install a fruit juice dispenser """

In [ ]:
juice_dispenser = BeverageDispenser(Juice())
install(juice_dispenser)

This is not legal because the cafeteria requires a dispenser that is specialzed for Juice.

In [ ]:
beverage_dispenser = BeverageDispenser(Beverage())
install(beverage_dispenser)

But this code is also illegal.

In [ ]:
orange_juice_dispenser = BeverageDispenser(OrangeJuice())
install(orange_juice_dispenser)

In [ ]:
!mypy invariant_demo.py

invariant_demo.py:7: error: Argument 1 to "install" has incompatible type "BeverageDispenser[Beverage]"; expected "BeverageDispenser[Juice]"  [arg-type]
invariant_demo.py:10: error: Argument 1 to "install" has incompatible type "BeverageDispenser[OrangeJuice]"; expected "BeverageDispenser[Juice]"  [arg-type]
Found 2 errors in 1 file (checked 1 source file)


In typing jargon, we say that `BeverageDispenser(Generic[T])` is invariant when `BeverageDispenser[OrangeJuice]` is not compatible with `BeverageDispenser[Juice]`

### Covariant Dispenser

In [ ]:
# covariant.py

from typing import TypeVar, Generic

class Beverage:
  """Any beverage."""

class Juice(Beverage):
  """Any fruit juice."""

class OrangeJuice(Juice):
  """orange juice."""

T_co = TypeVar('T_co', covariant=True)

class BeverageDispenser(Generic[T_co])
  def __init__(self, beverage: T_co) -> None:
    self.beverage = beverage

  def dispense(self) -> T_co:
    return self.beverage

def install(dispenser: BeverageDispenser[Juice]) -> None:
  """Install a fruit juice dispenser """

In [ ]:
!mypy covariant_demo.py

covariant_demo.py:7: error: Argument 1 to "install" has incompatible type "BeverageDispenser[Beverage]"; expected "BeverageDispenser[Juice]"  [arg-type]
Found 1 error in 1 file (checked 1 source file)


Summary of 'covariance': the subtype relationship of the parametrized dispensers varies in the same direction as the subtype relationship of the type parameters.

## Contravariant Trash Can

Note. For the sake of this example, let's make simplifying assumptions to classify trash in a neat hierarchy:
 - `Refuse` is the most general type of trash.
 - `Biodegradable` is a specific type of trash that can be decomposed by organisms over time. Some `Refuse` is not `Biodegradable`
 - `Compostable` is a specific type of `Biodegradable`. Not all `Biodegradable` is `Compostable` in our def.

In [1]:
# contravariant.py

from typing import TypeVar, Generic

class Refuse:
  """ Any refuse """

class Biodegradable(Refuse):
  """ Biodegradable refuse """

class Compostable(Biodegradable):
  """ Compostable refuse """

T_contra = TypeVar("T_contra", contravariant=True)

class TrashCan(Generic[T_contra]): # Trashcan is a contravariant on the type of refuse
  def put(self, refuse: T_contra) -> None:
    """ Store trash until dumped """

def deploy(trash_can: TrashCan[Biodegradable]):
  """ Deploy a trash can for bidegradable refuse """



In [2]:
!pip install mypy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 36.9 MB/s eta 0:00:00


In [ ]:
# contravariant_demo.py
from contravariant import Refuse, TrashCan, Compostable, Biodegradable, deploy

bio_can: TrashCan[Biodegradable] = TrashCan()
deploy(bio_can)

trash_can: TrashCan[Refuse] = TrashCan()
deploy(trash_can)

compost_can: TrashCan[Compostable] = TrashCan()
deploy(compost_can)

In [4]:
!mypy contravariant_demo.py

contravariant_demo.py:10: error: Argument 1 to "deploy" has incompatible type "TrashCan[Compostable]"; expected "TrashCan[Biodegradable]"  [arg-type]
Found 1 error in 1 file (checked 1 source file)


### Variance Review
 - A generic type `L` is invariant when there is no supertype or subtype relationship between two parametrized types, regardless of the relationship that may exist between the actual parameters.

 i.e., if `L` is invariant, then `L[A]` is not a supertype or a subtype of `L[B]`.

 As mentioned, Python's mutable collections are invariant by default. `list[int]` is not consistent with `list[flaot]`

 - __Covariant__: Consider two types `A` and `B`, where `B` is consistent-with `A`, and neither of them is `Any`. Some authors use the `<:` and `:>` symbols to denote type relationships.
 `A :> B`: A is a supertype-of or the same as B

 Given A :> B, a generic type `C` is covariant when C[A] :> C[B]

 > Immutable containers can be covariant. (e.g. `typing.FrozenSet`)

 - __Contravariant__: Given `A :> B`, a generic type `K` is contravariant when `K[A] <: K[B]`

 > write-only data structure, also known as a "sink"

 > `Callable[[ParamType, ...], ReturnType]` is contravariant on the parameter types, but covariant on the ReturnType

## Implementing a Generic Static Protocol

In [ ]:
# abs_demo.py

import math
from typing import NamedTuple, SupportsAbs

class Vector2d(NamedTuple):
  x: float
  y: float

  def __abs__(self) -> float: # defining __abs__ makes Vector2d consistent-with SupportsAbs
    return math.hypot(self.x, self.y)

def is_unit(v: SupportsAbs[float]) -> bool:
  return math.isclose(abs(v), 1.0)

assert issubclass(Vector2d, SupportsAbs)

v0 = Vector2d(0, 1)
sqrt2 = math.sqrt(2)
v1 = Vector2d(sqrt2 / 2, sqrt2 / 2)
v2 = Vector2d(1, 1)
v3 = complex(.5, math.sqrt(3) / 2)
v4 = 1 # int type is also consistent-with SupportsAbs

assert is_unit(v0)
assert is_unit(v1)
assert not is_unit(v2)
assert is_unit(v3)
assert is_unit(v4)


In [5]:
!mypy abs_demo.py

Success: no issues found in 1 source file


In [ ]:
from typing import Protocol, runtime_checkable, TypeVar

T_co = TypeVar('T_co', covariant=True)

@runtime_checkable
class RandomPicker(Protocol[T_co]):
  def pick(self) -> T_co: ...